In [1]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import statsmodels.api as sm
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.formula.api import ols
import nibabel as nib
from nilearn import plotting, image
import pingouin as pg
import yaml
from sklearn.metrics import roc_curve, auc, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

# Add project root to path
sys.path.append('..')

# Import project modules
from src.analysis.statistics import (
    compute_brain_volume_stats, 
    compare_groups, 
    correlate_imaging_clinical,
    longitudinal_analysis
)
from src.utils.metrics import classification_metrics, segmentation_metrics
from src.analysis.reporting import generate_statistical_report

# Load configuration
with open("../config.yml", 'r') as file:
    config = yaml.safe_load(file)

# Set the plotting style
sns.set(style="whitegrid")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['axes.titlesize'] = 16
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Define paths
data_dir = config['data']['processed_dir']
results_dir = config['analysis']['results_dir']
clinical_dir = config['data']['clinical_dir']
model_dir = config['models']['checkpoints_dir']

# Create results directory if it doesn't exist
os.makedirs(results_dir, exist_ok=True)

# Load the clinical data
print("Loading clinical data...")

# Subject information
adni_t1 = pd.read_csv(os.path.join(clinical_dir, 'ADNI_T1.csv'))
print(f"ADNI T1 dataset: {adni_t1.shape[0]} subjects")

# Clinical assessment data
cdr = pd.read_csv(os.path.join(clinical_dir, 'CDR.csv'))
mmse = pd.read_csv(os.path.join(clinical_dir, 'MMSE.csv'))
gdscale = pd.read_csv(os.path.join(clinical_dir, 'GDSCALE.csv'))
adas_adni1 = pd.read_csv(os.path.join(clinical_dir, 'ADAS_ADNI1.csv'))
adas_adnigo23 = pd.read_csv(os.path.join(clinical_dir, 'ADAS_ADNIGO23.csv'))
neurobat = pd.read_csv(os.path.join(clinical_dir, 'NEUROBAT.csv'))
demographics = pd.read_csv(os.path.join(clinical_dir, 'PTDEMOG.csv'))

# Display basic information about datasets
print(f"CDR dataset: {cdr.shape[0]} records")
print(f"MMSE dataset: {mmse.shape[0]} records")
print(f"GD Scale dataset: {gdscale.shape[0]} records")
print(f"ADAS ADNI1 dataset: {adas_adni1.shape[0]} records")
print(f"ADAS ADNIGO2/3 dataset: {adas_adnigo23.shape[0]} records")
print(f"Neurobat dataset: {neurobat.shape[0]} records")
print(f"Demographics dataset: {demographics.shape[0]} records")

# Load the segmentation and prediction results
print("\nLoading model prediction results...")
segmentation_results = pd.read_csv(os.path.join(results_dir, 'segmentation_volumes.csv'))
anomaly_results = pd.read_csv(os.path.join(results_dir, 'anomaly_detection_results.csv'))

# Merge model results with clinical data
print("Merging datasets...")
merged_data = (
    adni_t1
    .merge(segmentation_results, on='Subject_ID', how='inner')
    .merge(anomaly_results, on='Subject_ID', how='inner')
)

# Add key clinical measures
merged_data = (
    merged_data
    .merge(mmse[['Subject_ID', 'MMSE_Total']], on='Subject_ID', how='left')
    .merge(cdr[['Subject_ID', 'CDR_Global']], on='Subject_ID', how='left')
    .merge(demographics[['Subject_ID', 'Age', 'Gender', 'Education']], on='Subject_ID', how='left')
)

print(f"Final merged dataset: {merged_data.shape[0]} subjects with {merged_data.shape[1]} variables")

# Display distribution of diagnostic groups in the merged dataset
diagnosis_count = merged_data['Diagnosis'].value_counts()
print("\nDiagnostic group distribution in merged dataset:")
print(diagnosis_count)

plt.figure(figsize=(10, 6))
sns.countplot(x='Diagnosis', data=merged_data, palette='viridis')
plt.title('Distribution of Diagnostic Groups')
plt.xlabel('Diagnosis')
plt.ylabel('Count')
plt.xticks(rotation=0)
plt.tight_layout()
plt.savefig(os.path.join(results_dir, 'diagnosis_distribution.png'), dpi=300)
plt.show()

# Descriptive statistics of brain volumes by diagnostic group
print("\n1. Descriptive Statistics of Brain Volumes by Diagnostic Group")

# Select key brain structures
key_structures = ['Gray_Matter_Volume', 'White_Matter_Volume', 'Hippocampus_Volume',
                 'Ventricles_Volume', 'Total_Brain_Volume', 'Abnormality_Score']

# Group by diagnosis and compute stats
volume_stats = merged_data.groupby('Diagnosis')[key_structures].agg(['mean', 'std', 'min', 'max'])
print(volume_stats)

# Create boxplots
plt.figure(figsize=(18, 12))
for i, structure in enumerate(key_structures):
    plt.subplot(2, 3, i+1)
    sns.boxplot(x='Diagnosis', y=structure, data=merged_data, palette='viridis')
    plt.title(f'{structure} by Diagnosis')
    plt.xlabel('Diagnosis')
    plt.ylabel('Volume (mm³)' if 'Volume' in structure else 'Score')
    plt.xticks(rotation=0)
    
plt.tight_layout()
plt.savefig(os.path.join(results_dir, 'volume_boxplots.png'), dpi=300)
plt.show()

# 2. Statistical Comparisons Between Groups
print("\n2. Statistical Testing: Comparing Brain Volumes Between Diagnostic Groups")

# ANOVA for each brain structure
anova_results = {}
tukey_results = {}

for structure in key_structures:
    # ANOVA
    model = ols(f'{structure} ~ Diagnosis', data=merged_data).fit()
    anova_table = sm.stats.anova_lm(model, typ=2)
    anova_results[structure] = anova_table
    
    # Post-hoc Tukey test
    tukey = pairwise_tukeyhsd(endog=merged_data[structure], groups=merged_data['Diagnosis'], alpha=0.05)
    tukey_results[structure] = tukey
    
    # Print results
    print(f"\nANOVA for {structure}:")
    print(anova_table)
    
    print(f"\nTukey HSD for {structure}:")
    print(tukey)
    
    # Visualization
    plt.figure(figsize=(10, 6))
    sns.barplot(x='Diagnosis', y=structure, data=merged_data, palette='viridis', 
                ci=95, capsize=0.2)
    plt.title(f'{structure} Comparison (p={anova_table["PR(>F)"][0]:.4f})')
    plt.xlabel('Diagnosis')
    plt.ylabel('Volume (mm³)' if 'Volume' in structure else 'Score')
    
    # Add p-value annotations
    p_value = anova_table["PR(>F)"][0]
    significance = ""
    if p_value < 0.001:
        significance = "***"
    elif p_value < 0.01:
        significance = "**"
    elif p_value < 0.05:
        significance = "*"
        
    plt.text(0.5, 0.9, f'ANOVA: p={p_value:.4f} {significance}', 
             horizontalalignment='center', transform=plt.gca().transAxes)
    
    plt.tight_layout()
    plt.savefig(os.path.join(results_dir, f'{structure}_comparison.png'), dpi=300)
    plt.show()

# 3. Correlation Analysis: Brain Volumes and Clinical Scores
print("\n3. Correlation Analysis: Brain Volumes and Clinical Scores")

# Select the columns for correlation analysis
clinical_measures = ['MMSE_Total', 'CDR_Global', 'Age', 'Education']
correlation_vars = key_structures + clinical_measures

# Create correlation matrix
corr_matrix = merged_data[correlation_vars].corr(method='spearman')
print("Spearman correlation matrix:")
print(corr_matrix)

# Plot correlation heatmap
plt.figure(figsize=(14, 12))
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
cmap = sns.diverging_palette(230, 20, as_cmap=True)

sns.heatmap(corr_matrix, annot=True, mask=mask, cmap=cmap, vmax=1, vmin=-1, 
            center=0, square=True, linewidths=.5, fmt='.2f', cbar_kws={"shrink": .8})
plt.title('Spearman Correlation: Brain Volumes vs. Clinical Measures', fontsize=20)
plt.tight_layout()
plt.savefig(os.path.join(results_dir, 'correlation_heatmap.png'), dpi=300)
plt.show()

# Scatter plots for key relationships
key_correlations = [
    ('Hippocampus_Volume', 'MMSE_Total'),
    ('Gray_Matter_Volume', 'MMSE_Total'),
    ('Abnormality_Score', 'CDR_Global'),
    ('Total_Brain_Volume', 'Age')
]

plt.figure(figsize=(18, 12))
for i, (x_var, y_var) in enumerate(key_correlations):
    plt.subplot(2, 2, i+1)
    sns.scatterplot(x=x_var, y=y_var, hue='Diagnosis', data=merged_data, palette='viridis', alpha=0.7)
    
    # Calculate correlation and display it
    corr, p_val = stats.spearmanr(merged_data[x_var], merged_data[y_var], nan_policy='omit')
    significance = ""
    if p_val < 0.001:
        significance = "***"
    elif p_val < 0.01:
        significance = "**"
    elif p_val < 0.05:
        significance = "*"
        
    plt.title(f'{x_var} vs {y_var}\nr={corr:.2f}, p={p_val:.4f} {significance}')
    plt.xlabel(x_var)
    plt.ylabel(y_var)
    plt.legend(loc='best')
    
    # Add regression line
    sns.regplot(x=x_var, y=y_var, data=merged_data, scatter=False, 
                line_kws={'color': 'black', 'linestyle': '--'})
    
plt.tight_layout()
plt.savefig(os.path.join(results_dir, 'key_correlations.png'), dpi=300)
plt.show()

# 4. Classification Performance Analysis
print("\n4. Classification Performance Analysis")

# Evaluate model classification performance
y_true = pd.get_dummies(merged_data['Diagnosis'])
y_pred_proba = merged_data[['CN_Probability', 'MCI_Probability', 'AD_Probability']]

# Calculate ROC curves
plt.figure(figsize=(12, 10))
colors = ['blue', 'green', 'red']
classes = ['CN', 'MCI', 'AD']

for i, (cls, color) in enumerate(zip(classes, colors)):
    fpr, tpr, _ = roc_curve(y_true[cls], y_pred_proba[f'{cls}_Probability'])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, color=color, lw=2,
             label=f'ROC curve for {cls} (AUC = {roc_auc:.2f})')

plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves for Multi-class Classification')
plt.legend(loc="lower right")
plt.tight_layout()
plt.savefig(os.path.join(results_dir, 'roc_curves.png'), dpi=300)
plt.show()

# Confusion matrix
y_pred_class = merged_data['Predicted_Diagnosis']
y_true_class = merged_data['Diagnosis']
cm = confusion_matrix(y_true_class, y_pred_class)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

plt.figure(figsize=(10, 8))
sns.heatmap(cm_normalized, annot=True, cmap='Blues', fmt='.2f',
            xticklabels=classes, yticklabels=classes)
plt.title('Normalized Confusion Matrix')
plt.xlabel('Predicted Diagnosis')
plt.ylabel('True Diagnosis')
plt.tight_layout()
plt.savefig(os.path.join(results_dir, 'confusion_matrix.png'), dpi=300)
plt.show()

# Calculate classification metrics
accuracy = np.mean(y_pred_class == y_true_class)
print(f"Overall accuracy: {accuracy:.3f}")

from sklearn.metrics import classification_report
print("\nClassification Report:")
print(classification_report(y_true_class, y_pred_class))

# 5. Multivariate Analysis
print("\n5. Principal Component Analysis of Brain Volumes")

# Select features for PCA
pca_features = [col for col in merged_data.columns if 'Volume' in col]
X = merged_data[pca_features]

# Standardize the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Apply PCA
pca = PCA(n_components=2)
principal_components = pca.fit_transform(X_scaled)
pca_df = pd.DataFrame(data=principal_components, columns=['PC1', 'PC2'])
pca_df['Diagnosis'] = merged_data['Diagnosis']

# Visualize PCA results
plt.figure(figsize=(12, 10))
sns.scatterplot(x='PC1', y='PC2', hue='Diagnosis', data=pca_df, palette='viridis', s=100, alpha=0.7)
plt.title('PCA of Brain Volume Measurements')
plt.xlabel(f'Principal Component 1 ({pca.explained_variance_ratio_[0]:.2%} variance)')
plt.ylabel(f'Principal Component 2 ({pca.explained_variance_ratio_[1]:.2%} variance)')
plt.legend(title='Diagnosis')

# Add arrows for feature loadings
features = pca_features
loadings = pca.components_.T * np.sqrt(pca.explained_variance_)

for i, feature in enumerate(features):
    plt.arrow(0, 0, loadings[i, 0], loadings[i, 1], head_width=0.05, head_length=0.05, fc='gray', ec='gray')
    plt.text(loadings[i, 0] * 1.15, loadings[i, 1] * 1.15, feature, 
             color='black', ha='center', va='center', fontsize=10)

plt.grid(True)
plt.tight_layout()
plt.savefig(os.path.join(results_dir, 'pca_brain_volumes.png'), dpi=300)
plt.show()

# 6. Statistical Analysis of Anomaly Detection
print("\n6. Statistical Analysis of Anomaly Detection")

# Compare abnormality scores across diagnostic groups
plt.figure(figsize=(10, 6))
sns.boxplot(x='Diagnosis', y='Abnormality_Score', data=merged_data, palette='viridis')
plt.title('Abnormality Scores by Diagnostic Group')
plt.xlabel('Diagnosis')
plt.ylabel('Abnormality Score')
plt.tight_layout()
plt.savefig(os.path.join(results_dir, 'abnormality_scores.png'), dpi=300)
plt.show()

# ANOVA for abnormality scores
anova_model = ols('Abnormality_Score ~ Diagnosis', data=merged_data).fit()
anova_table = sm.stats.anova_lm(anova_model, typ=2)
print("ANOVA for Abnormality Scores:")
print(anova_table)

# Tukey's test for post-hoc analysis
tukey = pairwise_tukeyhsd(endog=merged_data['Abnormality_Score'], 
                         groups=merged_data['Diagnosis'], 
                         alpha=0.05)
print("\nTukey HSD for Abnormality Scores:")
print(tukey)

# Plot Tukey's test results
tukey_df = pd.DataFrame(data=tukey._results_table.data[1:], 
                       columns=tukey._results_table.data[0])
plt.figure(figsize=(12, 6))
sns.heatmap(pd.crosstab(tukey_df['group1'], tukey_df['group2'], 
                        values=tukey_df['p-adj'].astype(float), 
                        aggfunc='mean').fillna(1.0),
           annot=True, cmap='coolwarm_r', vmin=0, vmax=0.05)
plt.title("Tukey's HSD p-values for Abnormality Scores")
plt.tight_layout()
plt.savefig(os.path.join(results_dir, 'tukey_heatmap.png'), dpi=300)
plt.show()

ImportError: cannot import name 'compute_brain_volume_stats' from 'src.analysis.statistics' (D:\KLU 4th YEAR\Projects\Brain_AI\src\analysis\statistics.py)